# Fine-tuning to follow instructions

1. Dataset preparation
2. Fine-tuning the LLM
3. Evaluating LLM

## 1. Dataset preparation

Dataset consists of 1,100 instruction-response pairs, created for our case. 

In [25]:
import json
import os
import urllib

def download_and_load_file(file_path, url):
    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode('utf-8')
        with open(file_path, encoding='utf-8') as file:
            file.write(text_data)

    else: # skips downloading if file already exists
        with open(file_path, encoding='utf-8') as file:
            text_data = file.read()
    with open(file_path, encoding='utf-8') as file:
        data = json.load(file)
    
    return data

file_path = "instruction-data.json"
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)


data = download_and_load_file(file_path, url)
print(len(data))

1100


In [26]:
print("Example entry:\n", data[50])

Example entry:
 {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}


In [27]:
print("Another example entry:\n", data[999])

Another example entry:
 {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}


We will use Alpaca prompt style (other uses such as Phi-3 can be used)

In [28]:
# define the prompt formatt func

def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropiately completes the request."
        f"\n\n### Instruction\n{entry['instruction']}"
    )
    input_text = (
    f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    )
    return instruction_text + input_text

In [29]:
model_input = format_input(data[50])
desired_response = f"\n\n### Response:\n{data[50]['output']}"
print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropiately completes the request.

### Instruction
Identify the correct spelling of the following word.

### Input:
Ocassion

### Response:
The correct spelling is 'Occasion.'


In [30]:
model_input = format_input(data[999])
desired_response = f"\n\n### Response:\n{data[999]['output']}"
print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropiately completes the request.

### Instruction
What is an antonym of 'complicated'?

### Response:
An antonym of 'complicated' is 'simple'.


In [31]:
# partition the data

train_portion = int(len(data) * 0.85)
test_portion = int(len(data) * 0.1)
val_portion = len(data) - train_portion - test_portion

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

print("Training set length:", len(train_data))
print("Validation set length:", len(val_data))
print("Test set length:", len(test_data))


Training set length: 935
Validation set length: 55
Test set length: 110


Organizing data into training batches

In [ ]:
import torch
from torch.utils.data import Dataset

class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.enconded_text = []
        for entry in data:
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text
            self.enconded_text.append(
                tokenizer.encode(full_text)
            )
        
    def __get_item__(self, index):
        return self.encoded_texts[index]
    
    def __len__(self):
        return len(self.data)

In [32]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


In [ ]:
# we will padd now to match only the longest per batcj

def custom_collate_draft_1(batch, pad_token_id=50256, device="cpu"):
    batch_max_length = max(len(item) for item in batch)  # find longest seq in batch
    inputs_lst = []
    
    for item in batch: # pads and prepares input
        new_item = item.copy()
        new_item += [pad_token_id]

        padded = new_item + [pad_token_id] * (batch_max_length - len(new_item))
        inputs = torch.tensor(padded[:-1])  # removes extra padded token added earlier
        inputs_lst.append(inputs)
    
    inputs_tensor = torch.stack(inputs_lst).to(device) # converts list of inputs to single tensor
    return inputs_tensor